In [68]:
import urllib2, re, os
import bs4
import pandas as pd
from tqdm import tqdm_notebook

In [5]:
SVO_BASE_URL = 'http://svo2.cab.inta-csic.es/svo/theory/fps'

In [48]:
def generate_facilities_list():
    svo_html = urllib2.urlopen(SVO_BASE_URL).read()
    svo_soup = bs4.BeautifulSoup(svo_html)
    observatory_links = svo_soup.findAll('a', attrs={'href':re.compile('gname')})
    observatory_list = pd.DataFrame(index=range(len(observatory_links)), columns=['facility', 'url'])
    for i, obs_link in enumerate(observatory_links):
        observatory_list.loc[i] = obs_link.text, obs_link.attrs['href']
    return observatory_list

def generate_instruments_list(observatory_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    return observatory_soup.findAll('a', attrs={'href':re.compile('index\.php\?mode=browse&gname=.+?&gname2=.+?')})
#    filter_soup = observatory_soup.findAll('table')[8]
#    filter_table = pd.read_html(str(filter_soup), header=0)[0]
#    return filter_table

def generate_filter_list(instrument_url):
    observatory_html = urllib2.urlopen(observatory_url).read()
    observatory_soup = bs4.BeautifulSoup(observatory_html)
    filter_soup = observatory_soup.findAll('table')[8]
    filter_table = pd.read_html(str(filter_soup), header=0)[0]
    return filter_table

In [47]:
facilities_list = generate_facilities_list()

In [52]:
facility.url

'index.php?mode=browse&gname=2MASS'

In [70]:
tqdm_notebook?

In [71]:
all_filters = None
for i, facility in tqdm_notebook(facilities_list.iterrows(), desc='Facilities', total=len(facilities_list)):
    #print "Working on {0}".format(facility.facility)
    instruments = generate_instruments_list(os.path.join(SVO_BASE_URL, facility.url))
    if len(instruments) == 0:
        #print "{0} does not have separate instruments".format(facility.facility)
        filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, facility.url))
        if all_filters is None:
            all_filters = filter_list
        else:
            all_filters = all_filters.append(filter_list)
    else:
        for instrument in tqdm_notebook(instruments, desc='Instruments', leave=False):
            #print "Working on {0}".format(instrument.text)
            filter_list = generate_filter_list(os.path.join(SVO_BASE_URL, instrument.attrs['href']))
            if all_filters is None:
                all_filters = filter_list
            else:
                all_filters = all_filters.append(filter_list)


In [67]:
all_filters[all_filters['Filter ID'].str.startswith('AKARI')]

,Filter ID,Î»mean,Î»eff,Î»min,Î»max,Weff,ZP (Jy),Obs. Facility,Instrument,Description
0,AKARI/IRC.N2,24106.1,22684.3,16800,29497,6973.8,559.8,AKARI,IRC,IRC NIR N2
1,AKARI/IRC.N3,32761.3,31302.7,25370,39870,8605.4,325.5,AKARI,IRC,IRC NIR N3
2,AKARI/IRC.N4,44704.0,42505.2,33991,57514,15278.9,184.4,AKARI,IRC,IRC NIR N4
3,AKARI/IRC.S7,73059.1,69539.0,55236,87426,17325.8,72.5,AKARI,IRC,IRC MIR-S S7
4,AKARI/IRC.S9W,92182.0,82283.6,58461,121880,41027.1,47.5,AKARI,IRC,IRC MIR-S S9W
5,AKARI/IRC.S11,109489.1,101932.1,82698,152993,41042.6,33.7,AKARI,IRC,IRC MIR-S S11
6,AKARI/IRC.L15,161620.4,152310.5,122114,214681,59787.6,15.5,AKARI,IRC,IRC MIR-L L15
7,AKARI/IRC.L18W,198065.7,176094.9,136082,286691,99690.1,10.7,AKARI,IRC,IRC MIR-L L18W
8,AKARI/IRC.L24,233524.4,227526.4,184025,287037,53337.8,7.4,AKARI,IRC,IRC MIR-L L24


In [39]:
generate_instrument_list(os.path.join(SVO_BASE_URL, observatory_list.iloc[13, 1]))

[<a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Cape">Cape</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Cousins">Cousins</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=DDO">DDO</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Eggen">Eggen</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Geneva">Geneva</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=JHKLMN">JHKLMN</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Johnson">Johnson</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Stromgren">Stromgren</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Vilnius">Vilnius</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Walraven">Walraven</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=Washington">Washington</a>,
 <a href="index.php?mode=browse&amp;gname=GCPD&amp;gname2=WBVR">WBVR</a>]

'index.php?mode=browse&gname=Gemini&gname2=NIRI'

In [43]:
x = urllib2.urlopen('http://svo2.cab.inta-csic.es/svo/theory/fps/getdata.php?format=ascii&id=2MASS/2MASS.J').read()

In [44]:
x

'10620.0 0.0000000000\n10660.0 0.0004000000\n10700.0 0.0015000000\n10750.0 0.0027000001\n10780.0 0.0055000000\n10820.0 0.0122999996\n10840.0 0.0203000009\n10870.0 0.0306000002\n10890.0 0.0405000001\n10930.0 0.0515000001\n10960.0 0.0564000010\n11020.0 0.0718000010\n11050.0 0.2736000121\n11070.0 0.3409999907\n11090.0 0.3583999872\n11120.0 0.3801000118\n11160.0 0.3307000101\n11170.0 0.2395000011\n11200.0 0.2500999868\n11230.0 0.2833000124\n11280.0 0.2581999898\n11290.0 0.2515000105\n11320.0 0.5381000042\n11340.0 0.2231999934\n11380.0 0.5368999839\n11400.0 0.1102000028\n11430.0 0.5292000175\n11470.0 0.2619000077\n11540.0 0.3201999962\n11590.0 0.1743000001\n11640.0 0.6069999933\n11670.0 0.6179000139\n11700.0 0.6762999892\n11730.0 0.7279000282\n11750.0 0.7465000153\n11790.0 0.8303999901\n11820.0 0.7903000116\n11860.0 0.8095999956\n11880.0 0.8368999958\n11920.0 0.8360000253\n11950.0 0.7498999834\n11990.0 0.7080000043\n12020.0 0.6988000274\n12090.0 0.7049000263\n12160.0 0.7003999949\n12210.0 0